# My Learning from the notebook - 

- always forecast C classes when dealing with the classification problem
    - if y_out -> (B, C) and y_pred is -> (B,), this is perfect
- take care of the output dimensions always 

# We will train a NN (MLP) in this notebook



In [295]:
import pandas as pd
df_train = pd.read_csv("../../../data/classification/mobile_price_prediction/train.csv")

In [296]:
X = df_train.drop(columns = ["price_range"])
y = df_train["price_range"]

## doing train test split
the test data does not have labels so we only have the train data to play with

In [297]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
# Initialize the scaler
scaler = StandardScaler()

# Fit on training data and transform both train and test sets
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)



## turning the dataset into tensors now

In [298]:
import torch
import numpy as np
X_train = torch.from_numpy(np.asarray(X_train)).type(torch.float)
X_test = torch.from_numpy(np.asarray(X_test)).type(torch.float)
X_val = torch.from_numpy(np.asarray(X_val)).type(torch.float)

y_train = torch.from_numpy(np.asarray(y_train)).type(torch.long)
y_val = torch.from_numpy(np.asarray(y_val)).type(torch.long)
y_test = torch.from_numpy(np.asarray(y_test)).type(torch.long)


## Main part - Creating a model class (MLP)

[TensorFlow Playground – An Interactive Neural Network Explorer](https://playground.tensorflow.org/#activation=relu&batchSize=14&dataset=xor&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=10&networkShape=6,4,2,2,2,2&seed=0.36612&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&showTestData_hide=false&discretize_hide=false)

This interactive tool, introduced by the TensorFlow team in 2017, lets you **experiment with neural networks right in your browser**. You can tweak activation functions, network structure, data noise, and more—making it an excellent resource for both beginners and experienced practitioners to visually understand how neural networks learn.


In [302]:
# Import necessary module
from torch import nn
device = 'cpu'
from torch.nn.functional import normalize
# Define input and output layer dimensions
input_shape = X_train.shape[1]  # Number of input features
hidden_layer_shape = 200         # Number of neurons in the hidden layer
output_layer = 4             # Output dimension (e.g., regression output)

# Define a neural network model by subclassing nn.Module
class NN_model(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the first linear transformation (input to hidden layer)
        self.layer_1 = nn.Linear(in_features=input_shape, out_features=hidden_layer_shape)

        self.activation_1 = nn.ReLU()
        # Define the second linear transformation (hidden to output layer)
        self.layer_2 = nn.Linear(in_features=hidden_layer_shape, out_features=hidden_layer_shape)
        self.activation_2 = nn.ReLU()

        self.layer_3 = nn.Linear(in_features=hidden_layer_shape, out_features=output_layer)

        # self.activation = nn.Sigmoid()
    # Forward pass: defines how input data flows through the model
    def forward(self, x):
        # Pass data through layer 1, then through layer 2, and return the output
        x_hidden  = self.layer_1(x)
        x_hidden = self.activation_1 (x_hidden)
        x_hidden = self.layer_2(x_hidden)
        x_hidden = self.activation_2 (x_hidden)

        y_out = self.layer_3(x_hidden)
        
        # y_out_ = self.activation(y_out)
        # print("max", torch.max(y_out),
        #        "min", torch.min(y_out))

        return y_out


# Instantiate the model and move it to the specified device
model_0 = NN_model().to(device)
model_0

NN_model(
  (layer_1): Linear(in_features=20, out_features=200, bias=True)
  (activation_1): ReLU()
  (layer_2): Linear(in_features=200, out_features=200, bias=True)
  (activation_2): ReLU()
  (layer_3): Linear(in_features=200, out_features=4, bias=True)
)

## Setup a loss function

In [303]:
# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.CrossEntropyLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.Adam(model_0.parameters(), lr=0.01)

# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [304]:
epochs_nums = 100

for epoch in range(epochs_nums):

    model_0.train()
    
    y_logits = model_0(X_train)

    # 1. Forward pass (model outputs raw logits)
    # y_logits = y_logits.squeeze()

    y_pred = torch.argmax(y_logits, dim=1)

    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred)
    
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_val)
        test_pred = torch.argmax(test_logits, dim=1)
        # test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_val)
        test_acc = accuracy_fn(y_true=y_val,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")
        


    

Epoch: 0 | Loss: 1.39656, Accuracy: 21.72% | Test loss: 1.27231, Test acc: 42.19%
Epoch: 10 | Loss: 0.17753, Accuracy: 95.70% | Test loss: 0.26347, Test acc: 88.12%
Epoch: 20 | Loss: 0.01430, Accuracy: 100.00% | Test loss: 0.25825, Test acc: 90.31%
Epoch: 30 | Loss: 0.00171, Accuracy: 100.00% | Test loss: 0.34690, Test acc: 91.56%
Epoch: 40 | Loss: 0.00048, Accuracy: 100.00% | Test loss: 0.39399, Test acc: 90.00%
Epoch: 50 | Loss: 0.00025, Accuracy: 100.00% | Test loss: 0.41976, Test acc: 90.94%
Epoch: 60 | Loss: 0.00016, Accuracy: 100.00% | Test loss: 0.42133, Test acc: 90.62%
Epoch: 70 | Loss: 0.00013, Accuracy: 100.00% | Test loss: 0.43008, Test acc: 90.62%
Epoch: 80 | Loss: 0.00011, Accuracy: 100.00% | Test loss: 0.43209, Test acc: 90.94%
Epoch: 90 | Loss: 0.00010, Accuracy: 100.00% | Test loss: 0.43179, Test acc: 90.94%
